In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper


In [2]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
api_wrapper_arxie = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxie)
arxiv.name

'arxiv'

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
loader = WebBaseLoader("https://docs.ultralytics.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026C7E49A5C0>, search_kwargs={})

In [21]:
from langchain.tools.retriever import create_retriever_tool
retrieverl_tool = create_retriever_tool(retriever,"langsmith search","search any info about langsmith")
retrieverl_tool.name

'langsmith search'

In [22]:
tools = [wiki,arxiv,retrieverl_tool]

In [23]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\dhana\\Desktop\\genai_projects\\genai\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith search', description='search any info about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000026C275EB010>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026C7E49A5C

In [24]:
from langchain_groq import ChatGroq
import os
import openai

groq_api_key = os.getenv("GROQ_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [25]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

In [26]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [27]:
#agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [28]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [29]:
#agent executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [30]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [31]:
agent_executor.invoke({"input":"tell me about yolo"})




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'You Only Live Once'}`


Page: You Only Live Once (song)
Summary: "You Only Live Once" is song by American rock band the Strokes. It is the opening track on their third studio album, First Impressions of Earth. The song was written by Julian Casablancas with production from 
Invoking: `wikipedia` with `{'query': 'YOLO'}`




c:\Users\dhana\Desktop\genai_projects\genai\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\dhana\Desktop\genai_projects\genai\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


No good Wikipedia Search Result was found
Invoking: `wikipedia` with `{'query': 'You Only Live Once (YOLO)'}`




c:\Users\dhana\Desktop\genai_projects\genai\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\dhana\Desktop\genai_projects\genai\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


No good Wikipedia Search Result was found
Invoking: `wikipedia` with `{'query': 'YOLO'}`




c:\Users\dhana\Desktop\genai_projects\genai\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\dhana\Desktop\genai_projects\genai\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


No good Wikipedia Search Result was foundYOLO is actually an acronym for You Only Live Once, but it's also a popular real-time object detection system for videos and images. The system was introduced in a 2016 paper by Joseph Redmon and Ali Farhadi and is primarily designed for detecting objects in images and videos.

YOLO has several advantages over other object detection systems, including its speed and accuracy. It can process images in real-time, which makes it particularly useful for applications such as self-driving cars and surveillance systems.

> Finished chain.


{'input': 'tell me about yolo',
 'output': "YOLO is actually an acronym for You Only Live Once, but it's also a popular real-time object detection system for videos and images. The system was introduced in a 2016 paper by Joseph Redmon and Ali Farhadi and is primarily designed for detecting objects in images and videos.\n\nYOLO has several advantages over other object detection systems, including its speed and accuracy. It can process images in real-time, which makes it particularly useful for applications such as self-driving cars and surveillance systems."}

In [32]:
agent_executor.invoke({"input":"what is generative ai"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Generative AI'}`


Page: Generative artificial intelligence
Summary: Generative artificial intelligence (Generative AI, GenAI, or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. These mo
Invoking: `arxiv` with `{'query': 'Generative AI'}`


Published: 2025-02-16
Title: AI Generations: From AI 1.0 to AI 4.0
Authors: Jiahao Wu, Hengxu You, Jing Du
Summary: This paper proposes that Artificial Intelligence (AI) progresses through
several overlapping generations: AI 1.0 (Information AI), AI 
Invoking: `langsmith search` with `{'query': 'Generative AI'}`


YOLOv8 released in 2023 by Ultralytics, introduced new features and improvements for enhanced performance, flexibility, and efficiency, supporting a full range of vision AI tasks.
YOLOv9 introduces innovative methods like Programmable Gradient Information (PGI) and the Generalize

{'input': 'what is generative ai',
 'output': 'I will respond directly without using a tool.\n\nThe text you provided appears to be about a specific AI model called YOLO, its history, and its features. However, it does not seem to be directly related to the topic of Generative AI. Generative AI is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. It is used in applications such as content creation, data augmentation, and language translation.\n\nIf you would like to know more about Generative AI, I can provide you with some general information about the topic.'}

In [33]:
agent_executor.invoke({"input":"what is the research paper 1706.03762 about"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cThe research paper 1706.03762, titled "Attention Is All You Need", was published in 2017 and proposes a new approach to sequence transduction models, specifically in the area of machine translation. The paper introduces the Transformer model, which uses self-attention mechanisms to process input sequences, eliminating the need for traditional recurrent neural network (RNN) or convolutional neural network (CNN) based encoder-decoder architectures.

> Finished chain.


{'input': 'what is the research paper 1706.03762 about',
 'output': 'The research paper 1706.03762, titled "Attention Is All You Need", was published in 2017 and proposes a new approach to sequence transduction models, specifically in the area of machine translation. The paper introduces the Transformer model, which uses self-attention mechanisms to process input sequences, eliminating the need for traditional recurrent neural network (RNN) or convolutional neural network (CNN) based encoder-decoder architectures.'}

In [2]:
from pytube import YouTube

yt = YouTube("https://www.youtube.com/watch?v=p4pHsuEf4Ms")
if not yt.captions:
    print("No captions found for this video.")
    


HTTPError: HTTP Error 400: Bad Request

In [1]:
!pip install pytube